<a href="https://colab.research.google.com/github/ariadna-guzman/E3-Deteccion-de-plagio-TC3002B.301/blob/main/Evidencia_2_Modelo_Mejorado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Evidencia 2: Modelo Mejorado

- Ariadna Jocelyn Guzmán Jiménez A01749373
- Jorge Chávez Badillo A01749448
- Amy Murakami Tsutsumi A01750185


## Instalación de Librerías

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 97.7 MB/s eta 0:00:00


In [2]:
!pip -q install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.4 MB/s eta 0:00:00


In [3]:
!pip -q install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 14.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [4]:
!pip install Keras-Preprocessing

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.9 MB/s eta 0:00:00


## Importación de Librerías

In [5]:
# Kit de herramientas de lenguaje natural
import nltk 
from nltk.stem import PorterStemmer
from nltk.util import ngrams
import torch
# Para el cálculo de la distancia entre párrafos
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
# Para la manipulación y lectura de archivos
import os
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve
# Para la visualización, análisis y manipulación de los datos 
import pandas as pd
from tqdm import tqdm 
# Para la creación de vectores y matrices
import numpy as np
from keras_preprocessing.sequence import pad_sequences
# Implementación de BERT
from transformers import BertTokenizer, AutoModelForSequenceClassification
# Para la traducción de textos
from transformers import MarianMTModel, MarianTokenizer
from langdetect import detect, DetectorFactory

## Importación de Textos

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
genuinos = '/content/drive/Shareddrives/Desarrollo de Aplicaciones Avanzadas de Ciencias Computacionales/Reto/Evidencia 2/2023-Datasets/Etapa1/documentos-genuinos/'
sospechosos = '/content/drive/Shareddrives/Desarrollo de Aplicaciones Avanzadas de Ciencias Computacionales/Reto/Evidencia 2/2023-Datasets/Etapa1/docmentos-sospechosos/'

### Creación de Data Frame

In [9]:
def create_frame(path):
  '''
  Función que crea el dataframe que contiene el nombre del archivo 
  y contenido para la carpeta de archivos (path) seleccionada.
  '''
  content = []
  names = []
  files_list = sorted(os.listdir(path))
  for file_name in files_list:
    with open(path + file_name, 'r') as file:
      data = file.read().rstrip()
      content.append(data)
      names.append(file_name)
  dictionary = {'name': names, 'content': content}
  df = pd.DataFrame(dictionary)
  return df

Mostrar los primeros cinco documentos de la carpeta de documentos genuinos.

In [10]:
df_genuinos = create_frame(genuinos)
df_genuinos.head(5)

,name,content
0,org-300.txt,Automatic software plagiarism detection tools ...
1,org-301.txt,A casual comment by a student alerted the auth...
2,org-302.txt,Paraphrase types have been proposed by researc...
3,org-303.txt,This paper addresses the issue of text matchin...
4,org-304.txt,This work presents a Sentence Hashing Algorith...


## Implementación de Bert
***(Bidirectional Encoder Representations From Transformers)***

In [11]:
model_name = 'bert-base-uncased'

tokenizer = BertTokenizer.from_pretrained(model_name, do_lower_case = True)

model = AutoModelForSequenceClassification.from_pretrained(model_name,
                                                           output_attentions = False,
                                                           output_hidden_states = True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

### Vectorización

#### Por texto

In [14]:
def vectorize_text(tokenizer, model, abstract, MAX_LEN = 510):
  '''
  Función que genera la representación vectorial de un documento.
  Recibe como parámetros el tokenizer, el modelo, el documento y 
  el número máximo de tokens. 
  '''
  inputs = tokenizer.encode(abstract, 
                            add_special_tokens = True, 
                            max_length = MAX_LEN,)    

  results = pad_sequences([inputs], 
                          maxlen=MAX_LEN, 
                          dtype="long", 
                          truncating="post", 
                          padding="post")
  inputs = results[0] # Eliminar lista exterior

  # Máscaras
  ''' for i in inputs:
    mask = int(i>0)'''
  mask = [int(i > 0) for i in inputs]

  # Tensores
  inputs = torch.tensor(inputs)
  mask = torch.tensor(mask)

  # Dimensión adicional para batch
  inputs = inputs.unsqueeze(0)
  mask = mask.unsqueeze(0)

  # Evaluación de modelo
  model.eval()

  # Recopílación de estado a través de BERT
  with torch.no_grad():        
        logits, encoded_layers = model(input_ids = inputs, 
                                        token_type_ids = None, 
                                        attention_mask = mask,
                                        return_dict = False)

  layer_i = 12 # Última capa BERT antes de clasificación
  batch_i = 0 # Entrada del batch
  token_i = 0 # Primer token

  # Extraer el vector
  vector = encoded_layers[layer_i][batch_i][token_i]

  # Numpy array
  vector = vector.detach().cpu().numpy()

  return vector

#### Base de Datos

In [15]:
def vectorize_database(data):
  '''
  Función que genera una base de datos que contiene los vectores
  de todos los documentos. 
  '''
  vectors = []

  # Obtención de datos generales del contenido
  source = data.content.values

  # Recorrer texto para la obtención de los embeddings
  for content in tqdm(source):
    # Obtener el embedding
    vector = vectorize_text(tokenizer, model, content)
    vectors.append(vector)
  
  data['vectors'] = vectors
  data['vectors'] = data['vectors'].apply(lambda emb: np.array(emb))
  data['vectors'] = data['vectors'].apply(lambda emb: emb.reshape(1, -1))
  return data

### Creación de Vector para los Archivos Genuinos

In [16]:
vector_genuinos = vectorize_database(df_genuinos)

100%|██████████| 120/120 [04:24<00:00,  2.21s/it]


In [17]:
vector_genuinos.head()

,name,content,vectors
0,org-300.txt,Automatic software plagiarism detection tools ...,"[[-0.64271873, -0.26977772, -0.763381, -0.0089..."
1,org-301.txt,A casual comment by a student alerted the auth...,"[[-0.51203465, -0.3626315, -0.53556347, 0.1047..."
2,org-302.txt,Paraphrase types have been proposed by researc...,"[[-0.5537736, -0.32080397, -0.92781377, -0.348..."
3,org-303.txt,This paper addresses the issue of text matchin...,"[[-1.0950634, -0.34306043, -0.30031648, 0.2014..."
4,org-304.txt,This work presents a Sentence Hashing Algorith...,"[[-0.9064658, -0.37186188, -0.015655695, -0.31..."


## Funciones para la Detección de Plagio

In [18]:
def vectorize_cosine(sus_text):
  '''
  Función que crea un vector de acuerdo al texto de entrada 
  para ajustarlo para calcular la similitud de coseno.
  '''
  
  sus_vect = vectorize_text(tokenizer, model, sus_text)
  sus_vect = np.array(sus_vect)
  sus_vect = sus_vect.reshape(1, -1)
  return sus_vect

In [19]:
def detect_plagiarism(score, threshold):
  '''
  Función que determina si el texto es plagio o no de acuerdo 
  a un cierto umbral.
  '''

  status = False

  if(score >= threshold):
    status = True
  
  return status 

In [20]:
def plagiarism_analysis(sus_text, database_vector, threshold):
  '''
  Función que regresa los resultados de la detección de plagio, señalando
  los archivos comparados y el nivel de similitud obtenido.
  '''

  sus_vect = vectorize_cosine(sus_text)

  # Similitud Coseno
  database_vector['similarity'] = database_vector['vectors'].apply(
      lambda x: cosine_similarity(sus_vect, x))
  database_vector['similarity'] = database_vector['similarity'].apply(
      lambda x: x[0][0])
  
 # Top 5 Archivos similares  
  similar_files = database_vector.sort_values(by = 'similarity', 
                                              ascending = False)[0:4]
  df_result = similar_files[['name', 
                             'content', 
                             'similarity']].reset_index(drop = True)
  
  similarity_score = df_result.iloc[0]['similarity']
  most_similar_file = df_result.iloc[0]['content']
  status = detect_plagiarism(similarity_score, threshold)

  plagiarism_analysis = ('Archivo Sospechoso: \n' 
                         + sus_text
                         + '\nStatus: ' 
                         + str(status) 
                         + '\nSimilitud: ' 
                         + str(similarity_score) 
                         + '\nArchivo Similar: ' 
                         + most_similar_file)
  return plagiarism_analysis

## Detección de Plagio

In [21]:
df_sus = create_frame(sospechosos)
df_sus.head()

,name,content
0,FID-01.txt,This systematic review provides unique finding...
1,FID-02.txt,"In this review, we describe the application of..."
2,FID-03.txt,How to measure similarity between two sequence...
3,FID-04.txt,Text similarity measurement is the basis of na...
4,FID-05.txt,Hoy en día el idioma ha dejado de ser una barr...


In [22]:
df_sus.iloc[3]

name                                              FID-04.txt
content    Text similarity measurement is the basis of na...
Name: 3, dtype: object

In [23]:
for i in range(15):
  print('---------', i + 1, '---------',)
  sus_text = df_sus.iloc[i]['content']
  analysis = plagiarism_analysis(sus_text, vector_genuinos, 0.92)
  print(analysis)

--------- 1 ---------
Archivo Sospechoso: 
This systematic review provides unique findings with an updated examination of artificial intelligence (AI) in higher education (HE) from 2016 to 2022. Using the PRISMA principles and protocol, 138 articles were identified for a complete examination. Using a priori and reasoned coding, data from the 138 articles were extracted, analyzed and coded. The results of this study show that in 2021 and 2022, publications increased by almost two to three times the number of previous years. With this rapid increase in the number of AIEd HE publications, new trends have emerged. The results show that the research was carried out in six of the seven continents of the world. The trend shifted from the US to China, leading the number of publications. Another new trend is in the affiliation of researchers, as previous studies have shown a lack of researchers from education departments. This has now changed to be the more dominant department. Undergraduate st

## Funciones para el Análisis de Similitud

### Lemmatizer

In [ ]:
import nltk
nltk.download("wordnet")
nltk.download("omw-1.4")
from nltk.stem import WordNetLemmatizer

In [ ]:
#Función para aplicar lematización en los párrafos
def lemm_parrafo(parrafo):
  lemmatizer = WordNetLemmatizer()
  parrafo_lemm = []

  for palabra in parrafo: 
    palabra_lemm = lemmatizer.lemmatize(palabra)
    parrafo_lemm.append(palabra_lemm)

  parrafo_str = ' '.join(parrafo_lemm)
  return parrafo_str

### Stemming

In [ ]:
# Función para aplicar la herramienta de stemming a los párrafos
def stemm_parrafo(parrafo):
  stemmer = PorterStemmer()
  parrafo = parrafo.split()
  parrafo_stemm = []

  for palabra in parrafo:
    palabra_stem = stemmer.stem(palabra)
    parrafo_stemm.append(palabra_stem)

  parrafo_str = ' '.join(parrafo_stemm)
  return parrafo_str

## Conclusiones